# Задание 3

В базе данных вашей компании присутствуют следующие таблицы

**city**

* city_id – id города, где проходит акция
* client_city_id – id города, где находится покупатель
* city – город
* client_city – город, где находится покупатель

**client**

* client_id – id покупателя
* client_city_id – id города, где находится покупатель
* birth_date – дата рождения покупателя
* registration – дата регистрации покупателя

**promotion**

* promotion_id – id акции
* category_id – id категории
* promotion_name – акция
* category_name – категория
* partner_id – id партнёра
* partner_name – партнёр

**purchase**

* purchase_id – id покупки
* partner_id – id партнёра
* client_id – id покупателя
* city_id – id города
* promotion_id – id акции
* category_id – id категории
* purchase_date – дата покупки
* price – цена за единицу товара
* quantity – число проданных единиц
* status – статус покупки

Вам нужно написать запрос, чтобы получить такую таблицу: 

* purchase_date – дата покупки
* purchase_id – id покупки
* client_id – id покупателя
* client_age – возраст покупателя
* client_registration_age – как долго человек пользуется вашими магазинами
* client_category – содержит new или old, если это первая или последующая покупка соответственно
* promotion_name – акция
* category_name – категория
* partner_name – партнёр
* client_city – город, где находится покупатель
* city – город
* revenue – сумма выручки
* quantity – число проданных единиц

При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020

База данных доступна по имени project_variant_2.

In [1]:
import numpy as np
import pandas as pd
import pandahouse as ph

In [2]:
connection = dict(database='project_variant_2',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

In [3]:
query = """
SELECT 
    tb_purchase.purchase_date AS purchase_date, 
    tb_purchase.purchase_id AS purchase_id, 
    tb_purchase.client_id AS client_id, 
    
    dateDiff( 'year', tb_client.birth_date, now()) AS client_age, 
    dateDiff( 'day', tb_client.registration, now()) AS client_registration_age, 
    
    if( tb_first_purchase.purchase_date = purchase_date, 'new', 'old') AS client_category, 
    
    tb_promotion.promotion_name AS promotion_name, 
    tb_promotion.category_name AS category_name, 
    tb_promotion.partner_name AS partner_name, 
    
    tb_client_city.city AS client_city, 
    tb_purchase_city.city AS city, 
    
    toInt32(tb_purchase.price) * toInt32(tb_purchase.quantity) AS revenue, 
    toInt32(tb_purchase.quantity) AS quantity
    
-- tb_purchase    
FROM 
    project_variant_2.purchase as tb_purchase
    
-- tb_client   
LEFT JOIN 
    project_variant_2.client AS tb_client 
ON tb_purchase.client_id = tb_client.client_id

-- tb_first_purchase: for client_category
LEFT JOIN 
(
    SELECT
        client_id AS client_id, 
        MIN(purchase_date) AS purchase_date
    FROM project_variant_2.purchase AS tb_purchase
    GROUP BY tb_purchase.client_id
) AS tb_first_purchase
ON tb_purchase.client_id = tb_first_purchase.client_id

-- tb_client_city
LEFT JOIN 
( 
    SELECT 
    DISTINCT
        tb_city.client_city_id AS city_id, 
        tb_city.client_city AS city
    FROM 
        project_variant_2.city AS tb_city
) AS tb_client_city
ON tb_client.client_city_id = tb_client_city.city_id

-- tb_purchase_city
LEFT JOIN 
(
    SELECT
    DISTINCT
        tb_city.city_id AS city_id, 
        tb_city.city AS city
    FROM 
        project_variant_2.city as tb_city
) AS tb_purchase_city
ON tb_purchase.city_id = tb_purchase_city.city_id

-- tb_promotion
LEFT JOIN 
    project_variant_2.promotion AS tb_promotion
ON 
    tb_purchase.promotion_id = tb_promotion.promotion_id

WHERE 
    tb_purchase.status = 1 
    AND purchase_date >= '2020-05-01'
    AND purchase_date <= '2020-08-01'
"""

In [4]:
df = ph.read_clickhouse(query, connection=connection)

In [5]:
print(df.shape)
df.head()

(2735, 13)


,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-06-05,0,853,29,594,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Ярославль,Киев,150,5
1,2020-06-15,1,406,24,469,old,campaign_4,Прочее,KarpovCourses,Минск,Новгород,1150,5
2,2020-08-01,9,867,25,232,old,campaign_1,Бакалея,Google,Киев,Москва,1840,4
3,2020-07-16,15,109,30,795,old,campaign_4,Прочее,KarpovCourses,Щучинщина,Санкт-Петербург,440,8
4,2020-06-29,22,528,21,295,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Щучинщина,Щучинщина,390,2
